In [19]:
import os
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq

In [ ]:
os.environ['GROQ_API_KEY'] = "gsk......PTO"

In [21]:
import requests

# Assuming the PDF file is locally available in the /content/ directory
# We will use a file path instead of a URL.
file_path = "python_function.pdf"

# Read the PDF content directly from the file system
with open(file_path, "rb") as f:
    pdf_content = f.read()

# You can now work with the `pdf_content` variable
# as needed for further processing.
# ...

# If the file was intended to be accessed from a web server running locally,
# you'd need to provide the full URL including the scheme (e.g., http://localhost:8000/path/to/file.pdf).
# Replace 'http://localhost:8000' with the actual URL of your web server.
# url = "http://localhost:8000/content/The Rise of Generative AI in Creative Industries.pdf"
# response = requests.get(url)
# with open("/content/The Rise of Generative AI in Creative Industries.pdf", "wb") as f:
#     f.write(response.content)

In [22]:
loader = UnstructuredFileLoader('python_function.pdf')

In [23]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("python_function.pdf")
documents = loader.load()

Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 226 0 (offset 0)


In [24]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

In [25]:
type(texts)

list

In [26]:
len(texts)

25

In [27]:
texts[2]

Document(metadata={'source': 'python_function.pdf', 'page': 2}, page_content='Python and Types  \n•\u202f Dynamic typing: Python determines the data \ntypes of variable bindings in a program \nautomatically \n•\u202f Strong typing: But Python’s not casual about \ntypes, it enforces the types of objects     \n•\u202f For example, you can’t just append an integer \nto a string, but must first convert it to a string     \n x = “the answer is ”  # x bound to a string \n y = 23      # y bound to an integer. \n print x + y   # Python will complain!')

In [28]:
embeddings= HuggingFaceEmbeddings()

In [29]:
persist_dir = "doc_db"

In [30]:
vector_db = Chroma.from_documents(documents=texts,
                                 embedding=embeddings,
                                 persist_directory=persist_dir)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [31]:
retriever = vector_db.as_retriever()

In [32]:
# LLM from groq
llm = ChatGroq(
    model = "llama-3.1-8b-instant",
    temperature = 0)

In [33]:
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 return_source_documents=True)

In [34]:
query = "hello"
result = qa_chain.invoke({"query": query})
result["result"]

'It seems like you just said "hello". Is there something specific you\'d like to know or discuss?'